In [48]:
import pandas as pd
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
import tensorflow as tf
import math
import statistics
import keras
from tensorflow.python.ops import math_ops
import numpy as np


In [84]:
dfs_train = pd.read_csv('./pojutrze_24h_uzupelnianie_srednia_train.csv')
dfs_train = dfs_train.iloc[: , 1:]
dfs_test = pd.read_csv('./pojutrze_24h_test.csv')
dfs_test = dfs_test.iloc[: , 1:]
two_dfs = dfs_train.append(dfs_test, ignore_index=True)

In [86]:
dfs_train.describe()

,Latitude,Longitude,Humidity1_0,Pressure1_0,Temp1_0,WindDirSin1_0,WindDirCos1_0,WindSpeed1_0,DayOfYearSin1_0,DayOfYearCos1_0,...,Pressure3_0,Temp3_0,WindDirSin3_0,WindDirCos3_0,WindSpeed3_0,DayOfYearSin3_0,DayOfYearCos3_0,Temp5,Wind5_reg,Wind5_class
count,54612.000000,54612.000000,54612.000000,54612.000000,54612.000000,54612.000000,54612.000000,54612.000000,54612.000000,54612.000000,...,54612.000000,54612.000000,54612.000000,54612.000000,54612.000000,54612.000000,54612.000000,54612.000000,54612.000000,54612.000000
mean,-73.544668,37.066743,68.387529,1016.925132,288.460693,-0.085845,-0.005442,2.700937,-0.031231,0.017521,...,1016.931759,288.448984,-0.085882,-0.005318,2.701737,-0.030632,0.018587,288.437140,2.703624,0.035981
std,50.890928,5.734226,17.728223,14.449430,9.725213,0.495290,0.520757,1.526571,0.712276,0.700998,...,14.448791,9.726182,0.495452,0.520802,1.526357,0.711653,0.701630,9.726357,1.527193,0.186245
min,-123.119339,25.774269,7.708333,806.875000,248.462813,-1.000000,-1.000000,0.000000,-0.999991,-0.999963,...,806.875000,248.462813,-1.000000,-1.000000,0.000000,-0.999991,-0.999963,248.462813,0.000000,0.000000
25%,-105.401312,32.766126,59.250000,1012.333333,282.270769,-0.478081,-0.432890,1.583333,-0.746972,-0.683919,...,1012.333333,282.247000,-0.478347,-0.432917,1.583333,-0.746972,-0.683919,282.247000,1.583333,0.000000
50%,-86.471241,36.170429,70.708333,1017.625000,289.725354,-0.105204,0.005414,2.416667,-0.060213,0.055917,...,1017.666667,289.713021,-0.105252,0.005651,2.416667,-0.060213,0.055917,289.699502,2.416667,0.000000
75%,-74.874332,40.998211,80.416667,1024.083333,295.811680,0.274162,0.416068,3.500000,0.687053,0.702527,...,1024.083333,295.803021,0.274376,0.416280,3.500000,0.687053,0.702527,295.794271,3.500000,0.000000
max,35.216331,49.249660,100.000000,1098.500000,316.038000,0.999473,1.000000,15.208333,0.999991,1.000000,...,1098.500000,316.038000,0.999473,1.000000,15.208333,0.999991,1.000000,316.038000,15.208333,1.000000


In [87]:
dfs_test.describe()

,Latitude,Longitude,Humidity1_0,Pressure1_0,Temp1_0,WindDirSin1_0,WindDirCos1_0,WindSpeed1_0,DayOfYearSin1_0,DayOfYearCos1_0,...,Pressure3_0,Temp3_0,WindDirSin3_0,WindDirCos3_0,WindSpeed3_0,DayOfYearSin3_0,DayOfYearCos3_0,Temp5,Wind5_reg,Wind5_class
count,12383.000000,12383.000000,12383.000000,12383.000000,12383.000000,12383.000000,12383.000000,12383.000000,12383.000000,12383.000000,...,12383.000000,12383.000000,12383.000000,12383.000000,12383.000000,12383.000000,12383.000000,12383.000000,12383.000000,12383.000000
mean,-74.780988,37.170796,67.484724,1014.868587,289.143355,-0.084286,0.021304,3.162605,0.028209,-0.020975,...,1014.872513,289.159743,-0.084005,0.022486,3.157881,0.027370,-0.021930,289.179248,3.162033,0.043850
std,49.450114,5.724739,16.483144,11.058909,9.456267,0.473678,0.529921,1.400765,0.706863,0.706533,...,11.059852,9.448740,0.472927,0.530347,1.393661,0.707952,0.705446,9.434228,1.394486,0.204771
min,-123.119339,25.774269,11.166667,805.208333,248.813535,-0.989891,-0.991988,0.208333,-0.999991,-0.999963,...,805.208333,248.813535,-0.989891,-0.991988,0.208333,-0.999991,-0.999963,248.813535,0.208333,0.000000
25%,-104.984703,32.783058,59.000000,1011.708333,283.124167,-0.447850,-0.409562,2.166667,-0.693281,-0.720667,...,1011.708333,283.138542,-0.446902,-0.409246,2.166667,-0.693281,-0.720667,283.191229,2.166667,0.000000
50%,-86.784439,36.174969,70.166667,1015.916667,290.478250,-0.106784,0.024719,2.875000,0.060213,-0.047321,...,1015.916667,290.505000,-0.106519,0.027527,2.875000,0.060213,-0.047321,290.531250,2.875000,0.000000
75%,-75.163788,41.850029,79.000000,1019.916667,296.311000,0.252202,0.459202,3.875000,0.729558,0.677615,...,1019.916667,296.319583,0.251935,0.462018,3.875000,0.729558,0.677615,296.326042,3.875000,0.000000
max,35.216331,49.249660,100.000000,1041.833333,313.147083,0.993751,0.986479,18.458333,0.999991,1.000000,...,1041.833333,313.147083,0.993751,0.986479,18.458333,0.999991,1.000000,313.147083,18.458333,1.000000


In [50]:
print(dfs_train.shape)
print(dfs_test.shape)
print(two_dfs.shape)

(54612, 29)
(12383, 29)
(66995, 29)


### Prepare data for temperature regression

In [91]:
NUMBER_OF_Y_COLUMNS = 3
CHOSEN_Y_COLUMN = 3 # temp5

df_shuffle_train = dfs_train.sample(frac = 1)

Xs_train = df_shuffle_train.iloc[: , :-NUMBER_OF_Y_COLUMNS]
X_norm_train = Xs_train
X_norm_train = (Xs_train-Xs_train.mean())/Xs_train.std()
X_norm_train = X_norm_train.replace(np.nan, 0)


x_train = X_norm_train.iloc[: , :]
y_train = df_shuffle_train.iloc[: , -CHOSEN_Y_COLUMN:-(CHOSEN_Y_COLUMN - 1)]


df_shuffle_test = dfs_test.sample(frac = 1)

Xs_test = df_shuffle_test.iloc[: , :-NUMBER_OF_Y_COLUMNS]
X_norm_test = Xs_test
X_norm_test = (Xs_test-Xs_test.mean())/Xs_test.std()
X_norm_test = X_norm_test.replace(np.nan, 0)


x_test = X_norm_test.iloc[: , :]
y_test = df_shuffle_test.iloc[: , -CHOSEN_Y_COLUMN:-(CHOSEN_Y_COLUMN - 1)]

################################################################################################# wspolna standaryzacja na polaczonym zbiorze

# NUMBER_OF_Y_COLUMNS = 3
# CHOSEN_Y_COLUMN = 3 # temp5

# df_shuffle_train = dfs_train.sample(frac = 1)

# Xs_train = df_shuffle_train.iloc[: , :-NUMBER_OF_Y_COLUMNS]
# X_norm_train = Xs_train
# X_norm_train = (Xs_train-two_dfs.mean())/two_dfs.std()
# X_norm_train = X_norm_train.replace(np.nan, 0)


# x_train = X_norm_train.iloc[: , :]
# y_train = df_shuffle_train.iloc[: , -CHOSEN_Y_COLUMN:-(CHOSEN_Y_COLUMN - 1)]


# df_shuffle_test = dfs_test.sample(frac = 1)

# Xs_test = df_shuffle_test.iloc[: , :-NUMBER_OF_Y_COLUMNS]
# X_norm_test = Xs_test
# X_norm_test = (Xs_test-two_dfs.mean())/two_dfs.std()
# X_norm_test = X_norm_test.replace(np.nan, 0)


# x_test = X_norm_test.iloc[: , :]
# y_test = df_shuffle_test.iloc[: , -CHOSEN_Y_COLUMN:-(CHOSEN_Y_COLUMN - 1)]

############################################################################################## bez standaryzacji

# NUMBER_OF_Y_COLUMNS = 3
# CHOSEN_Y_COLUMN = 3 # temp5

# df_shuffle_train = dfs_train.sample(frac = 1)

# Xs_train = df_shuffle_train.iloc[: , :-NUMBER_OF_Y_COLUMNS]
# X_norm_train = Xs_train


# x_train = X_norm_train.iloc[: , :]
# y_train = df_shuffle_train.iloc[: , -CHOSEN_Y_COLUMN:-(CHOSEN_Y_COLUMN - 1)]


# df_shuffle_test = dfs_test.sample(frac = 1)

# Xs_test = df_shuffle_test.iloc[: , :-NUMBER_OF_Y_COLUMNS]
# X_norm_test = Xs_test


# x_test = X_norm_test.iloc[: , :]
# y_test = df_shuffle_test.iloc[: , -CHOSEN_Y_COLUMN:-(CHOSEN_Y_COLUMN - 1)]

################################################################################################## nie pamietam


# NUMBER_OF_Y_COLUMNS = 3
# CHOSEN_Y_COLUMN = 3 # temp5

# df_shuffle_train = dfs_train.sample(frac = 1)

# Xs_train = df_shuffle_train.iloc[: , :-NUMBER_OF_Y_COLUMNS]
# X_norm_train = Xs_train
# # X_norm_train = (Xs_train-Xs_train.mean())/Xs_train.std()
# # X_norm_train = X_norm_train.replace(np.nan, 0)


# x_train = X_norm_train.iloc[: , :]
# y_train = df_shuffle_train.iloc[: , -CHOSEN_Y_COLUMN:-(CHOSEN_Y_COLUMN - 1)]


# df_shuffle_test = dfs_test.sample(frac = 1)

# Xs_test = df_shuffle_test.iloc[: , :-NUMBER_OF_Y_COLUMNS]
# X_norm_test = Xs_test
# # X_norm_test = (Xs_test-Xs_test.mean())/Xs_test.std()
# # X_norm_test = X_norm_test.replace(np.nan, 0)


# x_test = X_norm_test.iloc[: , :]
# y_test = df_shuffle_test.iloc[: , -CHOSEN_Y_COLUMN:-(CHOSEN_Y_COLUMN - 1)]

######################################################################################################## podzial danych treningowych na test i train

# df_shuffle = dfs_train.sample(frac = 1)

# train_length = math.floor(df_shuffle.shape[0] * 0.8)

# Xs = df_shuffle.iloc[: , :-1]
# X_norm = Xs
# X_norm = (Xs-Xs.mean())/Xs.std()
# X_norm = X_norm.replace(np.nan, 0)

# x_test = X_norm.iloc[train_length: , :]
# y_test = df_shuffle.iloc[train_length: , -3:-2]
# x_train = X_norm.iloc[:train_length , :]
# y_train = df_shuffle.iloc[:train_length , -3:-2]

######################################################################################################### podzial danych testowych na test i train

# df_shuffle = dfs_test.sample(frac = 1)

# train_length = math.floor(df_shuffle.shape[0] * 0.8)

# Xs = df_shuffle.iloc[: , :-1]
# X_norm = Xs
# X_norm = (Xs-Xs.mean())/Xs.std()
# X_norm = X_norm.replace(np.nan, 0)

# x_test = X_norm.iloc[train_length: , :]
# y_test = df_shuffle.iloc[train_length: , -3:-2]
# x_train = X_norm.iloc[:train_length , :]
# y_train = df_shuffle.iloc[:train_length , -3:-2]

######################################################################################################### podzial danych polaczonych na test i train

# df_shuffle = two_dfs.sample(frac = 1)

# train_length = math.floor(df_shuffle.shape[0] * 0.8)

# Xs = df_shuffle.iloc[: , :-1]
# X_norm = Xs
# X_norm = (Xs-Xs.mean())/Xs.std()
# X_norm = X_norm.replace(np.nan, 0)

# x_test = X_norm.iloc[train_length: , :]
# y_test = df_shuffle.iloc[train_length: , -3:-2]
# x_train = X_norm.iloc[:train_length , :]
# y_train = df_shuffle.iloc[:train_length , -3:-2]

Regresja MSE temperatura 5000 danych

In [92]:
def correctly_predicted(predicted, y_test):
  counter = 0
  for i in range(len(predicted)):
    if abs(predicted[i] -  y_test[i]) <= 2:
      counter+=1

  return counter

def test_loss_function(loss):
  dim = x_test.shape[1]
  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate=0.1,
      decay_steps=100,
      decay_rate=0.9)
  adam = tf.keras.optimizers.Adam(
      learning_rate=lr_schedule)
  # adam = tf.keras.optimizers.Adam(
  #     learning_rate=0.00000001)
  model = Sequential()
  model.add(Dense(12, input_dim=dim, activation='relu'))
  model.add(Dense(10, activation='relu'))
  # model.add(Dropout(0.2))
  model.add(Dense(10, activation='relu'))
  # model.add(Dropout(0.2))
  model.add(Dense(1, activation='relu'))
  model.compile(loss=loss, optimizer=adam)
  model.fit(x=x_train, y=y_train, epochs=50, verbose=0)
  # model.fit(x=x_train, y=y_train, epochs=50)
  predicted = model.predict(x_test)
  return predicted

In [93]:
losses = [
          tf.keras.losses.MeanSquaredError(),
          tf.keras.losses.MeanAbsoluteError(),
          tf.keras.losses.Huber(),
          tf.keras.losses.LogCosh(),
          tf.keras.losses.MeanAbsolutePercentageError(),
          tf.keras.losses.MeanSquaredLogarithmicError(),
          tf.keras.losses.CosineSimilarity(),
          ]

predictions = []

for loss in losses:
  new_preds = test_loss_function(loss)
  predictions.append(new_preds)
  correct_predictions = correctly_predicted(new_preds.flatten(),  y_test.iloc[:,0].tolist())
  print(type(loss), ' finished')
  print(correct_predictions, correct_predictions / len(new_preds))

for i in range(len(predictions)):
  correct_predictions = correctly_predicted(predictions[i].flatten(),  y_test.iloc[:,0].tolist())
  print(type(losses[i]), '    ', correct_predictions, correct_predictions / len(predictions[i]))

<class 'keras.losses.MeanSquaredError'>  finished
5698 0.46014697569248164


KeyboardInterrupt: 

In [19]:
predictions[0].flatten().tolist()

[293.25140380859375,
 281.9801025390625,
 294.6438903808594,
 293.7368469238281,
 290.9010925292969,
 278.9808349609375,
 274.3131408691406,
 282.7371520996094,
 277.3113708496094,
 283.86907958984375,
 281.18536376953125,
 301.26251220703125,
 280.32183837890625,
 287.1481628417969,
 295.5872802734375,
 298.46075439453125,
 279.5885314941406,
 305.483154296875,
 306.1514587402344,
 275.5321960449219,
 288.9090576171875,
 288.5026550292969,
 272.60009765625,
 273.43145751953125,
 299.6044006347656,
 270.0516052246094,
 295.5876159667969,
 288.9927673339844,
 295.48675537109375,
 294.02178955078125,
 295.86529541015625,
 272.6702575683594,
 300.63812255859375,
 290.46990966796875,
 270.928955078125,
 280.01409912109375,
 280.5669250488281,
 279.4721374511719,
 288.3453674316406,
 291.1529541015625,
 272.0439147949219,
 299.5416259765625,
 303.18597412109375,
 286.09368896484375,
 276.363525390625,
 295.87451171875,
 272.5209045410156,
 295.66064453125,
 298.6722717285156,
 299.491790771

In [20]:
y_test.iloc[:,0].tolist()

[300.0770833333333,
 297.1383333333333,
 281.2341666666667,
 303.10375,
 298.82125,
 293.1954166666667,
 272.9629166666667,
 286.99741666666665,
 295.5175,
 290.14958333333334,
 271.38508333333334,
 296.25916666666666,
 278.629,
 289.78183333333334,
 303.2554166666667,
 287.145,
 286.9583333333333,
 295.2704166666667,
 301.6820833333333,
 291.87416666666667,
 300.36742396970834,
 299.1620833333333,
 273.33625,
 283.75125,
 302.44083333333333,
 285.41083333333336,
 293.21625,
 298.56041666666664,
 294.67541666666665,
 291.4025,
 283.935,
 264.5645833333333,
 298.41875,
 301.8768333333333,
 288.47625,
 292.1945833333333,
 284.66545833333333,
 296.4158333333333,
 297.7778333333333,
 289.89166666666665,
 283.68916666666667,
 285.40958333333333,
 277.68416666666667,
 289.95341666666667,
 293.64708333333334,
 296.04833333333335,
 283.90504166666665,
 286.53625,
 292.9044583333333,
 279.31291666666664,
 284.30833333333334,
 297.6775,
 288.06666666666666,
 303.33375,
 279.9225,
 284.8433333333

In [71]:
# porownanie z trzecim dniem

predictions = Xs_test.iloc[: , -6].tolist() # temp

correct_predictions = correctly_predicted(predictions,  y_test.iloc[:,0].tolist())
print('value from last measure', '    ', correct_predictions, correct_predictions / len(predictions))


value from last measure      6222 0.5024630541871922


In [ ]:
def custom_loss(y_true, y_pred):
  counter = 0
  for i in range(len(y_pred)):
    if abs(y_pred[i] -  y_true[i]) > 2:
      counter+=1
  return counter/len(y_pred)

In [ ]:
df_class_shuffle = dfs_class.sample(frac = 1)

train_class_length = math.floor(df_class_shuffle.shape[0] * 0.8)

Xs = df_class_shuffle.iloc[: , :-1]
X_norm = Xs
X_norm = (Xs-Xs.mean())/Xs.std()

x_test = X_norm.iloc[train_length: , 2:]
y_test = df_class_shuffle.iloc[train_length: , -1:]
x_train = X_norm.iloc[:train_length , 2:]
y_train = df_class_shuffle.iloc[:train_length , -1:]

#zamiana y na {0,1}
y_train.loc[y_train['WindSpeed5'] < 6.0, 'WindSpeed5'] = 0
y_train.loc[y_train['WindSpeed5'] >= 6.0, 'WindSpeed5'] = 1
y_test.loc[y_test['WindSpeed5'] < 6.0, 'WindSpeed5'] = 0
y_test.loc[y_test['WindSpeed5'] >= 6.0, 'WindSpeed5'] = 1

In [ ]:
print(y_train.head(20))

In [ ]:
dim = x_test.shape[1]
model = Sequential()
model.add(Dense(12, input_dim=dim, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x=x_train, y=y_train, epochs=50)